In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import time
from ipyexperiments import *
from torchsampler import ImbalancedDatasetSampler

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import gc
import torch 
import pandas as pd
import numpy as np
import time
import cProfile

In [3]:
from sklearn.metrics import precision_score,recall_score,roc_auc_score,confusion_matrix
import scikitplot as skplt

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [6]:
# def load_from_csv(csv_path):
#     df=pd.read_csv(csv_path)
#     val_index=df[df['Split']=='Val'].index().values()
#     test_index=

In [7]:
def csvlogger(cnf_matrix,precision,recall):
    tp=np.diag(cnf_matrix).astype('float')
    fp=cnf_matrix.sum(axis=0)-np.diag(cnf_matrix).astype('float')
    fn=cnf_matrix.sum(axis=1)-np.diag(cnf_matrix).astype('float')
    tn=cnf_matrix.sum()-(fp+fn+tp).astype('float')
    sensitivity=tp/(tp+fn)
    specificity=tn/(tn+fp)
    print(f'''recall:{recall}, precision:{precision}, 
                         Sensitivity:{sensitivity},Specificity:{specificity})''')
'''
#     try:
#         df.append(recall,precision,sensitivity,specificity) 
    
            
#     except NameError as e:
#         df=pd.DataFrame({'recall':recall, 'precision':precision, 
#                         'Sensitivity':sensitivity,'Specificity':specificity})
#     df.to_csv(f'{time.time()}-metrics-{image_size}.csv')
'''

"\n#     try:\n#         df.append(recall,precision,sensitivity,specificity) \n    \n            \n#     except NameError as e:\n#         df=pd.DataFrame({'recall':recall, 'precision':precision, \n#                         'Sensitivity':sensitivity,'Specificity':specificity})\n#     df.to_csv(f'{time.time()}-metrics-{image_size}.csv')\n"

In [12]:
image_size=64
num_epoch=20
input_path='/media/advenio/New Volume/New_data/final_data_subset/'

for i in range(3):
    with IPyExperimentsPytorch() as exp1:
        b_sizes=[256,64,5]
        weights=[64,128]
        
        model = models.resnet34(pretrained=False).cuda()
        model.fc = nn.Sequential(
                       nn.Linear(512, 256),
                       nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 3)).cuda()
        
        if i>0:
            model.load_state_dict(torch.load(f'./models/torch_cascade_Normal_Abnormal_{weights[i-1]}.h5'))
        else:
            pass
        
        image_size*=(2**i)
        
        batchsize=b_sizes[i]
        
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


        data_transforms = {
            'train':
            transforms.Compose([
                transforms.Resize((image_size,image_size)),
                transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize
            ]),
            'validation':
            transforms.Compose([
                transforms.Resize((image_size,image_size)),
                transforms.ToTensor(),
                normalize
            ]),
        }

        image_datasets = {
            'train': 
            datasets.ImageFolder(input_path + 'train', data_transforms['train']),
            'validation': 
            datasets.ImageFolder(input_path + 'val', data_transforms['validation'])
        }

        dataloaders = {
            'train':
            torch.utils.data.DataLoader(image_datasets['train'],
                                        batch_size=batchsize,
                                        shuffle=True,
                                        num_workers=0,),
                                       #sampler=ImbalancedDatasetSampler(image_datasets['train'])),  
            'validation':
            torch.utils.data.DataLoader(image_datasets['validation'],
                                        batch_size=batchsize,
                                        shuffle=True,
                                        num_workers=0,)
                                       #sampler=ImbalancedDatasetSampler(image_datasets['validation']))  
        }
        #weights=torch.tensor([(5492/19251),(13759/19251)])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.fc.parameters())

        def train_model(model, criterion, optimizer, num_epochs=3):
            accuracy=[]
            predictions=[]
            targets=[]
            for epoch in range(num_epochs):
                epoch_preds=[]
                epoch_labels=[]
                print('Epoch {}/{}'.format(epoch+1, num_epochs))
                print('-' * 10)

                for phase in ['train', 'validation']:
                    since=time.time()
                    if phase == 'train':
                        model.train()
                    else:
                        model.eval()

                    running_loss = 0.0
                    running_corrects = 0

                    for inputs, labels in dataloaders[phase]:
                        inputs = inputs.cuda()
                        labels = labels.cuda()

                        outputs = model(inputs)
                        outputs=outputs.cuda()
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()

                        _, preds = torch.max(outputs, 1)
                        running_loss += loss.item() * inputs.size(0)
                        running_corrects += torch.sum(preds == labels.data)
                        preds=list(map(int,preds))
                        labels=list(map(int,labels.data.cpu()))
                        epoch_preds.extend(preds)
                        epoch_labels.extend(labels)
                        predictions.extend(preds)
                        targets.extend(labels)
                    time_taken=(time.time()-since)/float(60)
                    epoch_loss = running_loss / len(image_datasets[phase])
                    epoch_acc = running_corrects.double() / len(image_datasets[phase])
                    precision=precision_score(epoch_labels,epoch_preds,average='macro')
                    recall=recall_score(epoch_labels,epoch_preds,average='macro')
                    #roc_score=roc_auc_score(epoch_labels,epoch_preds,average='micro')
                    print(f'Precision::{precision} Recall::{recall} ')

                    print('{} loss: {:.4f}, acc: {:.4f}, time: {}'.format(phase,
                                                                epoch_loss,
                                                                epoch_acc,time_taken))
            #if len(set(labels))==2:
            
            cnf_matrix=confusion_matrix(targets, predictions)
            skplt.metrics.plot_confusion_matrix(targets, predictions, normalize=False)
            plt.show()
            csvlogger(cnf_matrix,precision,recall)
            torch.cuda.empty_cache()
            gc.collect()
            #return model
            #torch.save(model.state_dict(),f'./models/torch_cascade_Normal_Abnormal_{image_size}.h5')
            return 0


        model_trained = train_model(model, criterion, optimizer, num_epochs=num_epoch)
            



*** Experiment started with the Pytorch backend


RuntimeError: CUDA error: device-side assert triggered

Error in callback <function CellLogger.post_run_cell at 0x7f5d53a99d08> (for post_run_cell):


RuntimeError: CUDA error: device-side assert triggered

**INFERNECE FOR Multi-class MODEL**

In [2]:
#Inference Script
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import time
from ipyexperiments import *
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import gc
import torch 
import pandas as pd
import numpy as np
import time
from torchvision import models 
from torch.nn import functional as F

In [6]:
model1 = models.resnet34(pretrained=False).cuda()
model1.fc = nn.Sequential(
               nn.Linear(512, 256),
               nn.ReLU(inplace=True),
    nn.Linear(256,128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2))
model1.load_state_dict(torch.load('/media/advenio/New Volume/models/torch_cascade_Normal_Abnormal_128.h5'))

<All keys matched successfully>

In [25]:
model1 = models.resnet34(pretrained=False).cuda()
model1.fc = nn.Sequential(
               nn.Linear(512, 256),
               nn.ReLU(inplace=True),
    nn.Linear(256,128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2)).cuda()
model2 = models.resnet50(pretrained=False).cuda()
model2.fc = nn.Sequential(
               nn.Linear(2048, 512),
               nn.ReLU(inplace=True),
            nn.Linear(512,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2)).cuda()
model1.load_state_dict(torch.load('/media/advenio/New Volume//models/torch_cascade_Normal_Abnormal_128.h5'))
model2.load_state_dict(torch.load('/media/advenio/New Volume//models/Torch_cascade_Subtle-Normal_weights_128.h5'))

<All keys matched successfully>

In [28]:
def inference(path):
    img=Image.open(path)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    image_size=128
    test_transforms=transforms.Compose([
                    transforms.Resize((image_size,image_size)),
                    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalize])
    img=test_transforms(img).cuda()
    img=torch.unsqueeze(img,0)
    #print(type(img))
    model1.eval()
    model2.eval()
    result=model1(img)
    prediction1=F.softmax(result).cpu().detach().numpy()
    
    if np.argmax(prediction1[0])==1:
        print('Abnormal')
    elif np.argmax(prediction1[0])==0:
        pred=model2(img)
        prediction2=F.softmax(pred).cpu().detach().numpy()
        if np.argmax(prediction2[0])==1:
            print('Subtle Normal')
        elif np.argmax(prediction2[0])==0 :
            print('Normal')

In [29]:
path='/media/advenio/UBUNTU 18_0/testing/'
for i in os.listdir(path):
    image_path=path+'/'+i
    inference(image_path)

/home/advenio/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Abnormal
Abnormal
Abnormal
Normal


/home/advenio/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Abnormal
Abnormal
Abnormal
Abnormal
Normal
Normal
Abnormal
Abnormal
Abnormal
Abnormal
Abnormal
Abnormal
Normal
Normal
Abnormal
Abnormal
Normal


In [31]:
def subtle(path):
    img=Image.open(path)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    image_size=128
    test_transforms=transforms.Compose([
                    transforms.Resize((image_size,image_size)),
                    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalize])
    img=test_transforms(img).cuda()
    img=torch.unsqueeze(img,0)
    pred=model2(img)
    prediction2=F.softmax(pred).cpu().detach().numpy()
    if np.argmax(prediction2[0])==1:
        print('Subtle Normal')
    elif np.argmax(prediction2[0])==0 :
        print('Normal')

In [32]:
path='/media/advenio/UBUNTU 18_0/testing/'
for i in os.listdir(path):
    image_path=path+'/'+i
    subtle(image_path)


/home/advenio/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal


In [ ]:
# img=Image.open('')
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
# image_size=128
# test_transforms=transforms.Compose([
#                 transforms.Resize((image_size,image_size)),
#                 transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ToTensor(),
#                 normalize])
# img=test_transforms(img).cuda()
# img=torch.unsqueeze(img,0)
# result=model1(img)
# prediction1=F.softmax(result).cpu().detach().numpy()
# prediction1
# np.argmax()
# if np.argmax(prediction1[0])==1:
#     print('Abnormal')
# elif np.argmax(k[0])==0:
#     pred=model2(img)
#     predicition2=F.softmax(result).cpu().detach().numpy()
#     if np.argmax(prediction2[0])==1:
#         print('Subtle Normal')
#     elif np.argmax(prediction2[0])==0 :
#         print('Normal')